In [13]:
# coding=utf-8
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

# 1. Get Yelp Data using API

In [3]:
CLIENT_ID = "3v45QlagqMHDo2gK_6Bdaw"

API_KEY = 'nV8HUn41_KVlL0dH2-zmFIrRyIvCLBvGIFBoj7WJ3_pAD5bypcRA-A97L1J7OOR31wJmlj__w6Nvngd1ihMNtrMWahbGmCjTBp_9EJWdzE6NF1amjgDMI_IAkYp-XHYx'

In [4]:
print(CLIENT_ID)
print(API_KEY)

3v45QlagqMHDo2gK_6Bdaw
nV8HUn41_KVlL0dH2-zmFIrRyIvCLBvGIFBoj7WJ3_pAD5bypcRA-A97L1J7OOR31wJmlj__w6Nvngd1ihMNtrMWahbGmCjTBp_9EJWdzE6NF1amjgDMI_IAkYp-XHYx


In [5]:
API_HOST = 'https://api.yelp.com' 
SEARCH_PATH = '/v3/businesses/search' 
BUSINESS_PATH = '/v3/businesses/'

In [6]:
def get_restaurants(api_key,location,number=50):
    import requests, json
    url = API_HOST + SEARCH_PATH
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    data = []
    for offset in range(0, 2000, 50):
        search_data = {
        'limit': number,
        'location': location.replace(' ', '+'),
        'term': "restaurant",
        'offset': offset
        }
    
        response = requests.request('GET', url, headers=headers, params=search_data)
        if response.status_code == 200:
            data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break

    return data

In [7]:
def get_business_review(api_key,business_id):
    import json
    import requests
    business_path = BUSINESS_PATH + business_id+"/reviews"
    url = API_HOST + business_path

    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }


    response = requests.request('GET', url, headers=headers).json()
   
    review_text = ''
    for review in response['reviews']:
        review_text += review['text']
    return review_text

In [8]:
df['Comments'] = df['alias'].apply(lambda x:get_business_review(API_KEY,x))

### To avoiding re-run the above codes which cost a long time, we save the updated dataset for the other notebooks.

In [9]:
# df.to_csv("02_rawdata_yelpAPI.csv", encoding='utf-8', index=False)

# 2. Get Michelin Restaurants List

In [14]:
# Washington,_D.C.  New_York_City   San_Francisco_Bay_Area   Chicago
def get_michelin(city):
    import requests
    from bs4 import BeautifulSoup
    import numpy as np
    import pandas as pd
    
    try:
        url = 'https://en.wikipedia.org/wiki/List_of_Michelin_starred_restaurants_in_%s'%(city)
        response = requests.get(url)
        if not response.status_code == 200: print('Error')

        results_page = BeautifulSoup(response.content,'lxml')
        restaurants = results_page.find_all('tr')
        michelin_list = []
        
        n=0
        for item in restaurants:
            if item.find('a').get('href') == "/wiki/Template:Lists_of_restaurants": break
            try: 
                # get name
                rest_name = item.find('td').get_text().strip()
                # get neighbor
                rest_neighbor = item.find_all('td')[1].get_text().strip()

                # get link in Michelin
                #rest_link = 'https://guide.michelin.com/us/%s/%s/restaurant'%(city,name)
                # chicago, new-york, san-francisco, washington-dc
                # keiko-a-nob-hill --- Keiko à Nob Hill            
                
                # get stars
                stars = item.find_all('td')[2:]
                rest_star = []
                for i in stars:
                    try: star = int(i.find('img').get('alt')[0])
                    except: 
                        if i.get_text()[:6]=='Closed': rest_star = [];break
                        else: star=0
                    rest_star.append(star)
                
                if rest_star == []: continue    
                rest_info = [rest_name,rest_neighbor]
                rest_info.extend(rest_star)
                michelin_list.append(rest_info)
                n=n+1

            except:
                part=n  # 0 and 60
                first_year = item.find_all('th')[2].get_text()[:4].strip()
                if part>0: 
                    michelin_data0=pd.DataFrame(michelin_list,columns=column)                    
                    years=[str(i) for i in range(int(first_year),2020-len(column[2:]))]
                    column = ['name','borough']; column.extend(years)
                    continue

                years=[str(i) for i in range(int(first_year),2020)]
                column = ['name','borough']
                column.extend(years)

        michelin_data=pd.DataFrame(michelin_list[part:],columns=column)

        if part>0: 
            michelin_data = pd.merge(michelin_data, michelin_data0, how='right', sort=True, on='name', suffixes=('', '_del'))
            michelin_data['borough']=michelin_data['borough_del']
            michelin_data = michelin_data.drop(['borough_del'],axis=1).fillna(0)
        
        michelin_data['city'] = city                
    except: return None
    
    return michelin_data

In [15]:
ny=get_michelin('New_York_City') 
sf = get_michelin('San_Francisco_Bay_Area') 
ch=get_michelin('Chicago')
dc = get_michelin('Washington,_D.C.') 

mi_df = pd.concat([ny,sf,ch,dc]).fillna(0)
mi_df_name=mi_df['name']
mi_df = mi_df.drop(['name'],axis=1)
mi_df.insert(0,'Name',mi_df_name)
mi_df.index = [np.arange(0,len(mi_df))]

# mi_df.to_excel('01_Get_Michelin_List.xlsx')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


# 3. Data Preprocessing

In [10]:
# df = pd.read_csv("02_rawdata_yelpAPI.csv", ignore_index = True)
# df.head()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
alias            5000 non-null object
categories       5000 non-null object
coordinates      5000 non-null object
display_phone    4871 non-null object
distance         5000 non-null float64
id               5000 non-null object
image_url        4997 non-null object
is_closed        5000 non-null bool
location         5000 non-null object
name             5000 non-null object
phone            4871 non-null float64
price            4618 non-null object
rating           5000 non-null float64
review_count     5000 non-null int64
transactions     5000 non-null object
url              5000 non-null object
Comments         5000 non-null object
dtypes: bool(1), float64(3), int64(1), object(12)
memory usage: 630.0+ KB


Create city column

In [5]:
df['location'][0]

"{'address1': '95 1st Ave', 'address2': '', 'address3': '', 'city': 'New York', 'zip_code': '10003', 'country': 'US', 'state': 'NY', 'display_address': ['95 1st Ave', 'New York, NY 10003']}"

In [6]:
import ast
ast.literal_eval(df['location'][0])['city']

'New York'

In [7]:
def get_city(loc_string):
    return ast.literal_eval(loc_string)['city']

df['city'] = df['location'].apply(get_city)
df['city'].unique()

array(['New York', 'Brooklyn', 'Long Island City', 'Sunnyside',
       'Elmhurst', 'Astoria', 'Downtown Brooklyn', 'Ridgewood',
       'Rego Park', 'Woodside', 'Queens', 'Jackson Heights',
       'Jersey City', 'Brooklyn Heights', 'Chicago', 'Washington, DC',
       'Arlington', 'Washington DC', 'Bethesda', 'Hyattsville',
       'Falls Church', 'Glenarden', 'Annandale', 'McLean', 'Takoma Park',
       'Silver Spring', 'Lanham', 'Alexandria', 'Chevy Chase',
       'National Harbor', 'Oakland', 'Lafayette', 'Berkeley',
       'Emeryville', 'Alameda', 'San Leandro', 'Albany', 'Orinda',
       'Moraga', 'Berekely', 'Castro Valley', 'San  Leandro', 'Richmond',
       'Los Angeles', 'West Hollywood', 'North Hollywood',
       'Beverly Hills', 'Glendale', 'Hollywood', 'Huntington Park',
       'Universal City'], dtype=object)

Drop the original review column

In [10]:
df = df.drop(['Comments'], axis=1)

Drop all rows that have price == Nan

In [11]:
df = df[df['price'].notnull()]
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,city
0,upstate-new-york-2,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 40.7263139141197, 'longitude': -7...",(646) 791-5400,2411.918718,ehUuSk5gPTCQmwS_ubgKRA,https://s3-media1.fl.yelpcdn.com/bphoto/RvnUVU...,False,"{'address1': '95 1st Ave', 'address2': '', 'ad...",Upstate,1.646792e+10,$$,4.5,1699,['restaurant_reservation'],https://www.yelp.com/biz/upstate-new-york-2?ad...,New York
1,amélie-new-york,"[{'alias': 'french', 'title': 'French'}, {'ali...","{'latitude': 40.7327, 'longitude': -73.99766}",(212) 533-2962,3038.869903,ETgJqJHV7BW6pIr9Ox74sA,https://s3-media4.fl.yelpcdn.com/bphoto/cSDgVu...,False,"{'address1': '22 W 8th St', 'address2': '', 'a...",Amélie,1.212533e+10,$$,4.5,2428,['restaurant_reservation'],https://www.yelp.com/biz/am%C3%A9lie-new-york?...,New York
2,uglyduckling-brooklyn,"[{'alias': 'tradamerican', 'title': 'American ...","{'latitude': 40.686023, 'longitude': -73.991302}",(718) 451-3825,2174.283882,CwOAKJdX8AMz5iAoA-ZEuA,https://s3-media2.fl.yelpcdn.com/bphoto/vyH8WZ...,False,"{'address1': '166 Smith St', 'address2': '', '...",Uglyduckling,1.718451e+10,$$,4.0,432,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/uglyduckling-brooklyn...,Brooklyn
3,aunt-jakes-new-york,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 40.7190315, 'longitude': -73.9977...",(646) 858-0470,1538.062738,I2lgw_7DUnwD92ND4PN-Ow,https://s3-media2.fl.yelpcdn.com/bphoto/hMhn_l...,False,"{'address1': '149 Mulberry St', 'address2': No...",Aunt Jake's,1.646858e+10,$$,4.0,1143,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/aunt-jakes-new-york?a...,New York
4,lovemama-new-york,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...","{'latitude': 40.7303859, 'longitude': -73.9860...",(212) 254-5370,2858.772941,jjJc_CrkB2HodEinB6cWww,https://s3-media1.fl.yelpcdn.com/bphoto/bLlFKT...,False,"{'address1': '174 2nd Ave', 'address2': '', 'a...",LoveMama,1.212255e+10,$$,4.0,4136,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/lovemama-new-york?adj...,New York


Create latitude, longtitude columns

In [12]:
df['coordinates'][0]

"{'latitude': 40.7263139141197, 'longitude': -73.9864901976935}"

In [13]:
ast.literal_eval(df['coordinates'][0])['latitude']

40.7263139141197

In [14]:
def get_lat(coor_string):
    return ast.literal_eval(coor_string)['latitude']

def get_long(coor_string):
    return ast.literal_eval(coor_string)['longitude']

df['latitude'] = df['coordinates'].apply(get_lat)
df['longitude'] = df['coordinates'].apply(get_long)

In [15]:
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,city,latitude,longitude
0,upstate-new-york-2,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 40.7263139141197, 'longitude': -7...",(646) 791-5400,2411.918718,ehUuSk5gPTCQmwS_ubgKRA,https://s3-media1.fl.yelpcdn.com/bphoto/RvnUVU...,False,"{'address1': '95 1st Ave', 'address2': '', 'ad...",Upstate,1.646792e+10,$$,4.5,1699,['restaurant_reservation'],https://www.yelp.com/biz/upstate-new-york-2?ad...,New York,40.726314,-73.986490
1,amélie-new-york,"[{'alias': 'french', 'title': 'French'}, {'ali...","{'latitude': 40.7327, 'longitude': -73.99766}",(212) 533-2962,3038.869903,ETgJqJHV7BW6pIr9Ox74sA,https://s3-media4.fl.yelpcdn.com/bphoto/cSDgVu...,False,"{'address1': '22 W 8th St', 'address2': '', 'a...",Amélie,1.212533e+10,$$,4.5,2428,['restaurant_reservation'],https://www.yelp.com/biz/am%C3%A9lie-new-york?...,New York,40.732700,-73.997660
2,uglyduckling-brooklyn,"[{'alias': 'tradamerican', 'title': 'American ...","{'latitude': 40.686023, 'longitude': -73.991302}",(718) 451-3825,2174.283882,CwOAKJdX8AMz5iAoA-ZEuA,https://s3-media2.fl.yelpcdn.com/bphoto/vyH8WZ...,False,"{'address1': '166 Smith St', 'address2': '', '...",Uglyduckling,1.718451e+10,$$,4.0,432,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/uglyduckling-brooklyn...,Brooklyn,40.686023,-73.991302
3,aunt-jakes-new-york,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 40.7190315, 'longitude': -73.9977...",(646) 858-0470,1538.062738,I2lgw_7DUnwD92ND4PN-Ow,https://s3-media2.fl.yelpcdn.com/bphoto/hMhn_l...,False,"{'address1': '149 Mulberry St', 'address2': No...",Aunt Jake's,1.646858e+10,$$,4.0,1143,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/aunt-jakes-new-york?a...,New York,40.719031,-73.997716
4,lovemama-new-york,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...","{'latitude': 40.7303859, 'longitude': -73.9860...",(212) 254-5370,2858.772941,jjJc_CrkB2HodEinB6cWww,https://s3-media1.fl.yelpcdn.com/bphoto/bLlFKT...,False,"{'address1': '174 2nd Ave', 'address2': '', 'a...",LoveMama,1.212255e+10,$$,4.0,4136,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/lovemama-new-york?adj...,New York,40.730386,-73.986061


Clean category

In [16]:
print(df['categories'][0])
type(df['categories'][0])

[{'alias': 'seafood', 'title': 'Seafood'}, {'alias': 'wine_bars', 'title': 'Wine Bars'}, {'alias': 'beerbar', 'title': 'Beer Bar'}]


str

In [17]:
ast.literal_eval(df['categories'][0])[2]

def get_cat_list(cat_string):
    cat_list = []
    for cat_dict in ast.literal_eval(cat_string):   
        cat_list.append(cat_dict['title'])
    return cat_list


df['category'] = df['categories'].apply(get_cat_list)


Convert price from dollar signs to numbers

In [19]:
def price_to_number(dollar_sign):
    if dollar_sign == '$':
        price = 1
    elif dollar_sign == '$$':
        price = 2
    elif dollar_sign == '$$$':
        price = 3
    elif dollar_sign == '$$$$':
        price = 4
    return price

df['price_level'] = df['price'].apply(price_to_number)

Convert transaction to three dummy variables restaurant_reservation, delivery, pickup

In [20]:
print(df['transactions'][0])
type(df['transactions'][0])

['restaurant_reservation']


str

In [21]:
'restaurant_reservation' in ast.literal_eval(df['transactions'][0])

True

In [22]:
def delivery(transac):
    if 'delivery' in ast.literal_eval(transac):
        return 1
    else:
        return 0
    
def pickup(transac):
    if 'pickup' in ast.literal_eval(transac):
        return 1
    else:
        return 0
    
def reserve(transac):
    if 'restaurant_reservation' in ast.literal_eval(transac):
        return 1
    else:
        return 0
    
df['delivery'] = df['transactions'].apply(delivery)
df['pickup'] = df['transactions'].apply(pickup)
df['reservation'] = df['transactions'].apply(reserve)

df.head()


,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,...,transactions,url,city,latitude,longitude,category,price_level,delivery,pickup,reservation
0,upstate-new-york-2,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 40.7263139141197, 'longitude': -7...",(646) 791-5400,2411.918718,ehUuSk5gPTCQmwS_ubgKRA,https://s3-media1.fl.yelpcdn.com/bphoto/RvnUVU...,False,"{'address1': '95 1st Ave', 'address2': '', 'ad...",Upstate,...,['restaurant_reservation'],https://www.yelp.com/biz/upstate-new-york-2?ad...,New York,40.726314,-73.986490,"[Seafood, Wine Bars, Beer Bar]",2,0,0,1
1,amélie-new-york,"[{'alias': 'french', 'title': 'French'}, {'ali...","{'latitude': 40.7327, 'longitude': -73.99766}",(212) 533-2962,3038.869903,ETgJqJHV7BW6pIr9Ox74sA,https://s3-media4.fl.yelpcdn.com/bphoto/cSDgVu...,False,"{'address1': '22 W 8th St', 'address2': '', 'a...",Amélie,...,['restaurant_reservation'],https://www.yelp.com/biz/am%C3%A9lie-new-york?...,New York,40.732700,-73.997660,"[French, Wine Bars]",2,0,0,1
2,uglyduckling-brooklyn,"[{'alias': 'tradamerican', 'title': 'American ...","{'latitude': 40.686023, 'longitude': -73.991302}",(718) 451-3825,2174.283882,CwOAKJdX8AMz5iAoA-ZEuA,https://s3-media2.fl.yelpcdn.com/bphoto/vyH8WZ...,False,"{'address1': '166 Smith St', 'address2': '', '...",Uglyduckling,...,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/uglyduckling-brooklyn...,Brooklyn,40.686023,-73.991302,"[American (Traditional), Gastropubs]",2,1,1,1
3,aunt-jakes-new-york,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 40.7190315, 'longitude': -73.9977...",(646) 858-0470,1538.062738,I2lgw_7DUnwD92ND4PN-Ow,https://s3-media2.fl.yelpcdn.com/bphoto/hMhn_l...,False,"{'address1': '149 Mulberry St', 'address2': No...",Aunt Jake's,...,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/aunt-jakes-new-york?a...,New York,40.719031,-73.997716,"[Italian, Wine Bars, Pasta Shops]",2,1,1,1
4,lovemama-new-york,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...","{'latitude': 40.7303859, 'longitude': -73.9860...",(212) 254-5370,2858.772941,jjJc_CrkB2HodEinB6cWww,https://s3-media1.fl.yelpcdn.com/bphoto/bLlFKT...,False,"{'address1': '174 2nd Ave', 'address2': '', 'a...",LoveMama,...,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/lovemama-new-york?adj...,New York,40.730386,-73.986061,"[Thai, Malaysian, Vietnamese]",2,1,1,1


Reorder columns

In [23]:
cols = df.columns.tolist()
cols

['alias',
 'categories',
 'coordinates',
 'display_phone',
 'distance',
 'id',
 'image_url',
 'is_closed',
 'location',
 'name',
 'phone',
 'price',
 'rating',
 'review_count',
 'transactions',
 'url',
 'city',
 'latitude',
 'longitude',
 'category',
 'price_level',
 'delivery',
 'pickup',
 'reservation']

In [24]:
cols = [cols[9]] + [cols[-5]] + [cols[-8]] + cols[-7:-5] + [cols[-4]] + cols[12:14] + cols[-3:] + cols[:9] + cols[10:12] + cols[14:16]
cols

['name',
 'category',
 'city',
 'latitude',
 'longitude',
 'price_level',
 'rating',
 'review_count',
 'delivery',
 'pickup',
 'reservation',
 'alias',
 'categories',
 'coordinates',
 'display_phone',
 'distance',
 'id',
 'image_url',
 'is_closed',
 'location',
 'phone',
 'price',
 'transactions',
 'url']

In [25]:
df = df[cols]
df.head()

,name,category,city,latitude,longitude,price_level,rating,review_count,delivery,pickup,...,display_phone,distance,id,image_url,is_closed,location,phone,price,transactions,url
0,Upstate,"[Seafood, Wine Bars, Beer Bar]",New York,40.726314,-73.986490,2,4.5,1699,0,0,...,(646) 791-5400,2411.918718,ehUuSk5gPTCQmwS_ubgKRA,https://s3-media1.fl.yelpcdn.com/bphoto/RvnUVU...,False,"{'address1': '95 1st Ave', 'address2': '', 'ad...",1.646792e+10,$$,['restaurant_reservation'],https://www.yelp.com/biz/upstate-new-york-2?ad...
1,Amélie,"[French, Wine Bars]",New York,40.732700,-73.997660,2,4.5,2428,0,0,...,(212) 533-2962,3038.869903,ETgJqJHV7BW6pIr9Ox74sA,https://s3-media4.fl.yelpcdn.com/bphoto/cSDgVu...,False,"{'address1': '22 W 8th St', 'address2': '', 'a...",1.212533e+10,$$,['restaurant_reservation'],https://www.yelp.com/biz/am%C3%A9lie-new-york?...
2,Uglyduckling,"[American (Traditional), Gastropubs]",Brooklyn,40.686023,-73.991302,2,4.0,432,1,1,...,(718) 451-3825,2174.283882,CwOAKJdX8AMz5iAoA-ZEuA,https://s3-media2.fl.yelpcdn.com/bphoto/vyH8WZ...,False,"{'address1': '166 Smith St', 'address2': '', '...",1.718451e+10,$$,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/uglyduckling-brooklyn...
3,Aunt Jake's,"[Italian, Wine Bars, Pasta Shops]",New York,40.719031,-73.997716,2,4.0,1143,1,1,...,(646) 858-0470,1538.062738,I2lgw_7DUnwD92ND4PN-Ow,https://s3-media2.fl.yelpcdn.com/bphoto/hMhn_l...,False,"{'address1': '149 Mulberry St', 'address2': No...",1.646858e+10,$$,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/aunt-jakes-new-york?a...
4,LoveMama,"[Thai, Malaysian, Vietnamese]",New York,40.730386,-73.986061,2,4.0,4136,1,1,...,(212) 254-5370,2858.772941,jjJc_CrkB2HodEinB6cWww,https://s3-media1.fl.yelpcdn.com/bphoto/bLlFKT...,False,"{'address1': '174 2nd Ave', 'address2': '', 'a...",1.212255e+10,$$,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/lovemama-new-york?adj...


In [2]:
# df.to_csv('01-1_Yelp_data_cleaned.csv')